In [9]:
### Import libraries
import openai
import os
import time
from datetime import datetime
import pandas as pd
#import numpy as np
from numpy.linalg import norm
import psycopg2
import clingo
from clingo.control import Control
import json
from yaml import load, Loader
from helper import *
from stories import *
#import multiprocessing
import random

In [10]:
conn_string = "dbname=thesis user=postgres password=postgres"
conn = psycopg2.connect(conn_string)
print("Connection established")

Connection established


### There are 2 tables of results: panel and ne_panel

In [11]:
sql = '''WITH ranked_files AS (
    SELECT 
        file,
        story,
        encoding,
        version,
        kpi,
        jaccard,
        ROW_NUMBER() OVER (PARTITION BY story ORDER BY kpi DESC) AS rn
    FROM 
        panel
    WHERE 
        jaccard < 1
    AND
        version = 10
)
SELECT 
    file,
    story,
    encoding,
    version,
    kpi,
    jaccard
FROM 
    ranked_files
WHERE 
    rn = 1
AND
    version = 10
;''' 
cursor = conn.cursor()
cursor.execute(sql)
result = cursor.fetchall()
conn.commit()
cursor.close()

In [12]:
conn.commit()
cursor.close()

In [13]:
df = pd.DataFrame(result,columns=["file","story","encoding","version","kpi","jaccard"])

In [14]:
df

,file,story,encoding,version,kpi,jaccard
0,creek_from_minotaur_and_sudoku_v10_20231203.lp,creek,% The possible cells are d...,10,53.4,0.60
1,hop_from_minotaur_and_lights_v10_20231204.lp,hop,% Define fields at the com...,10,15.2,0.20
2,lights_from_creek_and_sudoku_v10_20231204.lp,lights,% Define the possible posi...,10,57.7,0.27
3,minotaur_from_hop_and_yosenabe_v10_20231203.lp,minotaur,% The player starts at the...,10,-6.9,0.08
4,seeknumbers_from_hop_and_sudoku_v10_20231203.lp,seeknumbers,% Initial parameters\ndire...,10,21.6,0.33
5,sudoku_from_seeknumbers_and_hop_v10_20231203.lp,sudoku,"% Generate cells\ncell(X,Y...",10,41.8,0.77
6,yosenabe_from_hop_and_seeknumbers_v10_20231204.lp,yosenabe,% Define the neighbours of...,10,27.7,0.40


In [15]:
for idx, row in df.iterrows():
    with open(os.path.join("selected",row["file"]),"w") as file:
        file.writelines(row["encoding"])    